# 스마트 정수장 알고리즘 시계열기반 추출 학습: 약품 공정

### 1. 필요한 Python 라이브러리 Import 하기

In [ ]:
import numpy as np
import pandas as pd
import datetime
import scipy
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import r_regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge

# RandomForestRegressor를 임포트합니다.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pickle
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import platform

if platform.system() == "Darwin":  #
    plt.rc("font", family="AppleGothic")
else:
    plt.rc("font", family="NanumGothic")

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",  # ttf 파일이 저장되어 있는 경로
    name="NanumGothic",
)  # 원하는 폰트 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가

plt.rcParams.update({"font.size": 18, "font.family": "NanumGothic"})  # 폰트 설정

plt.rcParams["axes.unicode_minus"] = False

### 2. 전처리된 데이터 불러오기

In [ ]:
load_df = pd.read_csv("SN_total.csv")
load_df = load_df.set_index("시간")
load_df.index = pd.DatetimeIndex(load_df.index)
raw_df = load_df.copy()
raw_df.head(5)

### 3. 변수 추가 (Log)

In [ ]:
df = raw_df.copy()

In [ ]:
df["로그 원수 탁도"] = np.log10(df["원수 탁도"])
df["로그 응집제 주입률"] = np.log10(df["3단계 1계열 응집제 주입률"])
df

In [ ]:
X = df[
    [
        "로그 원수 탁도",
        "원수 pH",
        "원수 알칼리도",
        "원수 전기전도도",
        "원수 수온",
        "3단계 원수 유입 유량",
        "3단계 침전지 체류시간",
    ]
]
y = df["로그 응집제 주입률"]
Xt, Xts, yt, yts = train_test_split(X, y, test_size=0.2, shuffle=False)

### 4. Random Forest 모델 개발

In [ ]:
regressor = RandomForestRegressor(random_state=2, n_jobs=-1)

params = {
    "min_samples_leaf": [6],
    "max_depth": [20],
    "n_estimators": [100],
}

rscv = RandomizedSearchCV(regressor, params, n_iter=10)
rscv.fit(Xt, yt)
model = rscv.best_estimator_
model

### 5. 모델 성능평가

In [ ]:
yt_pred = model.predict(Xt)
yts_pred = model.predict(Xts)

mse_train = mean_squared_error(10**yt, 10**yt_pred)
mse_test = mean_squared_error(10**yts, 10**yts_pred)
print(f"학습 데이터 MSE: {mse_train}")
print(f"테스트 데이터 MSE: {mse_test}")

r2_train = r2_score(10**yt, 10**yt_pred)
r2_test = r2_score(10**yts, 10**yts_pred)
print(f"학습 데이터 R2: {r2_train}")
print(f"테스트 데이터 R2: {r2_test}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ax = axes[0]
ax.scatter(Xt["로그 원수 탁도"], yt, s=3, label="학습 데이터 (실제)")
ax.scatter(Xt["로그 원수 탁도"], yt_pred, s=3, label="학습 데이터 (예측)", c="r")
ax.grid()
ax.legend(fontsize=13)
ax.set_xlabel("로그 원수 탁도")
ax.set_ylabel("로그 응집제 주입률")
ax.set_title(
    rf"학습 데이터  MSE: {round(mse_train, 4)}, $R^2$: {round(r2_train, 2)}",
    fontsize=18,
)

ax = axes[1]
ax.scatter(Xts["로그 원수 탁도"], yts, s=3, label="테스트 데이터 (실제)")
ax.scatter(Xts["로그 원수 탁도"], yts_pred, s=3, label="테스트 데이터 (예측)", c="r")
ax.grid()
ax.legend(fontsize=13)
ax.set_xlabel("로그 원수 탁도")
ax.set_ylabel("로그 응집제 주입률")
ax.set_title(
    rf"테스트 데이터  MSE: {round(mse_test, 4)}, $R^2$: {round(r2_test, 2)}",
    fontsize=18,
)

### 6. RF 저장 및 불러와서 사용하기

In [ ]:
with open("rf_ex2.pickle", "wb") as f:
    pickle.dump(rscv, f)

In [ ]:
with open("rf_ex2.pickle", "rb") as f:
    rf_model = pickle.load(f)

In [ ]:
from sklearn.metrics import mean_squared_error

# 테스트 세트의 레이블을 예측합니다.
y_pred = rf_model.predict(Xts)

# rmse를 계산합니다.
mse_test = mean_squared_error(10**yts, 10**y_pred)

# rmse를 출력합니다.
print("테스트 세트 점수: {:.3f}".format(mse_test))